```
Topic:        Project 1 MTA
Subject:      Exploring changes in MTA data during Yankees Games
Date:         07/05/2020
Group:        Una Bayasgalan, Jacky Lu, Isaac Wang
```

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
def get_data(week_nums):
    '''
    Takes in a list of days representing consecutive weeks. Returns MTA turnstile data for that timeframe.
    
        Parameters:
            week_nums (list): days written in 2 digit year/month/day 2 format
        
        Returns:
            (DataFrame): MTA turnstile data for that timeframe
    '''
    
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

In [3]:
# These weeks were chosen because they represent the entirety of the Yankees baseball season.        
week_nums = [190323, 190330, 190406, 190413, 190420, 190427, 190504, 190511, 190518, 190525, 
             190601, 190608, 190615, 190622, 190629, 190706, 190713, 190720, 190727, 190803,
             190810, 190817, 190824, 190831, 190907, 190914, 190921, 190928, 191005, 191012,
             191019]
turnstiles_df = get_data(week_nums)

In [4]:
turnstiles_df = get_data(week_nums)

# save as a pickle object to reduce loading times
with open('pickled_data/raw_data_df.pickle', 'wb') as to_write:
    pickle.dump(turnstiles_df, to_write)

In [2]:
# open pickle object so get_data doesn't have to run again
with open('pickled_data/raw_data_df.pickle', 'rb') as read_file:
    turnstiles_df = pickle.load(read_file)

In [3]:
# Clean up column names
turnstiles_df.columns = turnstiles_df.columns.str.strip()

In [4]:
# Create DataFrame taking first entry and exit for each turnstile per day.
daily_df = (turnstiles_df
            .groupby(["C/A","UNIT","SCP","STATION","DATE"])[["ENTRIES","EXITS"]]
            .first()
            .rename(columns={"ENTRIES":"FIRST_ENTRIES", "EXITS":"FIRST_EXITS"})   
           )
daily_df.head()

FIRST_ENTRIES  FIRST_EXITS
C/A  UNIT SCP      STATION DATE                                  
A002 R051 02-00-00 59 ST   03/16/2019        6980704      2367517
                           03/17/2019        6981564      2367731
                           03/18/2019        6982134      2367864
                           03/19/2019        6983646      2368374
                           03/20/2019        6985159      2368888

In [5]:
# Add column for the next day's entry and exit for each turnstile per day.
daily_df["NEXT_DAY_ENTRIES"] = (daily_df
                                .groupby(["C/A","UNIT","SCP","STATION"])["FIRST_ENTRIES"]
                                .apply(lambda group: group.shift(-1))
                                )
daily_df["NEXT_DAY_EXITS"] = (daily_df
                              .groupby(["C/A","UNIT","SCP","STATION"])["FIRST_EXITS"]
                              .apply(lambda group: group.shift(-1))
                             )
daily_df.head()

FIRST_ENTRIES  FIRST_EXITS  \
C/A  UNIT SCP      STATION DATE                                     
A002 R051 02-00-00 59 ST   03/16/2019        6980704      2367517   
                           03/17/2019        6981564      2367731   
                           03/18/2019        6982134      2367864   
                           03/19/2019        6983646      2368374   
                           03/20/2019        6985159      2368888   

                                       NEXT_DAY_ENTRIES  NEXT_DAY_EXITS  
C/A  UNIT SCP      STATION DATE                                          
A002 R051 02-00-00 59 ST   03/16/2019         6981564.0       2367731.0  
                           03/17/2019         6982134.0       2367864.0  
                           03/18/2019         6983646.0       2368374.0  
                           03/19/2019         6985159.0       2368888.0  
                           03/20/2019         6986742.0       2369347.0

Calculate daily entries and exits for each day by subtracting the current day's entries from the next day's entry.

Additionally, I will mark negative counts with null values. Also, any counts higher than a threshold will be marked with null values.

Assuming 1 turnstile entry per second, there can be a max of 86,400 turnstiles entries per day.

In [6]:
def daily_count(row, movement, threshold):
    '''
    Calculates the number of daily entries or exits per turnstile per day. Counts above 
    threshold and negative counts are marked as null.
    
        Parameters:
            row (Series): row of turnstile data
            movement (str): specifying ENTRIES or EXITS
            threshold (int): maximum number of turnstile entries allowed per day
        
        Returns:
            daily_count (float/null): number of daily entries or exits
    '''
    
    
    first_movement = "FIRST_" + movement
    next_movement = "NEXT_DAY_" + movement
    
    # mark daily counts as null when there is no next day data
    if pd.isnull(row[next_movement]):
        daily_count = np.nan
    else:
        daily_count = row[next_movement] - row[first_movement]
        
    # mark negative and higher than threshold daily entries as null    
    if daily_count < 0:
        daily_count = np.nan
    elif daily_count > threshold:
        daily_count = np.nan
        
    return daily_count

In [7]:
daily_df = daily_df.reset_index()
daily_df["DAILY_ENTRIES"] = (daily_df
                                  .apply(daily_count, 
                                         movement="ENTRIES", 
                                         threshold=86400,
                                         axis=1
                                        )
                                 )
daily_df["DAILY_EXITS"] = (daily_df
                                  .apply(daily_count, 
                                         movement="EXITS", 
                                         threshold=86400,
                                         axis=1)
                                 )
daily_df.head()

,C/A,UNIT,SCP,STATION,DATE,FIRST_ENTRIES,FIRST_EXITS,NEXT_DAY_ENTRIES,NEXT_DAY_EXITS,DAILY_ENTRIES,DAILY_EXITS
0,A002,R051,02-00-00,59 ST,03/16/2019,6980704,2367517,6981564.0,2367731.0,860.0,214.0
1,A002,R051,02-00-00,59 ST,03/17/2019,6981564,2367731,6982134.0,2367864.0,570.0,133.0
2,A002,R051,02-00-00,59 ST,03/18/2019,6982134,2367864,6983646.0,2368374.0,1512.0,510.0
3,A002,R051,02-00-00,59 ST,03/19/2019,6983646,2368374,6985159.0,2368888.0,1513.0,514.0
4,A002,R051,02-00-00,59 ST,03/20/2019,6985159,2368888,6986742.0,2369347.0,1583.0,459.0


In [11]:
# Calculate number of rows before and after dropping null values
print(daily_df.shape)
daily_df.dropna(subset=["DAILY_ENTRIES","DAILY_EXITS"], inplace=True)
print(daily_df.shape) # 1055371 - 1036462 = 18909 rows (1.8% of original dataset) was dropped

(1055371, 11)
(1036462, 11)


In [12]:
# Add in a datetime object in preparation for merging with date data.
daily_df["DAY_OF_YEAR"] = pd.to_datetime(daily_df["DATE"])
daily_df.sample(5)

,C/A,UNIT,SCP,STATION,DATE,FIRST_ENTRIES,FIRST_EXITS,NEXT_DAY_ENTRIES,NEXT_DAY_EXITS,DAILY_ENTRIES,DAILY_EXITS,DAY_OF_YEAR
444103,N334B,R341,00-03-01,75 AV,09/12/2019,753914,242241,753937.0,242294.0,23.0,53.0,2019-09-12
45502,A050,R088,00-00-01,CORTLANDT ST,10/15/2019,4020759,4190082,4021089.0,4190550.0,330.0,468.0,2019-10-15
969833,R528,R097,00-03-01,JUNCTION BLVD,08/28/2019,8671985,7886492,8673902.0,7888330.0,1917.0,1838.0,2019-08-28
744833,R170,R191,00-00-01,103 ST,08/09/2019,13524064,3962123,13525979.0,3962589.0,1915.0,466.0,2019-08-09
338743,N123B,R439,01-05-01,ROCKAWAY AV,04/12/2019,0,79,0.0,79.0,0.0,0.0,2019-04-12


In [20]:
with open('pickled_data/yankee_cleaned_df.pickle', 'wb') as to_write:
    pickle.dump(daily_df, to_write)

We have 2 external data sources that we need to merge into our dataset. First, we need to merge our dataset with  
the dates the Yankee's had a home game.

Data was obtained from [MLB's website](https://www.mlb.com/yankees/schedule/2019/fullseason).  
These dates were transcribed onto [Google Sheets](https://docs.google.com/spreadsheets/d/1jrYnZN8lfO_pnJ8AFaxkOnoni_EdLOO7Vj3RHeImaMc/edit#gid=89819907) and formatted to match MTA dates. Then the data was finally transformed into the following list using Excel.

In [21]:
home_game = ["3/28/2019", "3/30/2019", "3/31/2019", "4/1/2019", "4/2/2019", "4/3/2019", "4/12/2019", "4/13/2019", "4/14/2019",
             "4/16/2019", "4/17/2019", "4/18/2019", "4/19/2019", "4/20/2019", "4/21/2019", "5/3/2019", "5/4/2019", "5/5/2019",
             "5/6/2019", "5/7/2019", "5/8/2019","5/9/2019","5/15/2019","5/17/2019","5/18/2019","5/19/2019","5/27/2019",
             "5/28/2019","5/29/2019","5/31/2019","6/1/2019","6/2/2019","6/11/2019","6/17/2019","6/18/2019","6/19/2019",
             "6/20/2019","6/21/2019","6/22/2019","6/23/2019","6/24/2019","6/25/2019","6/26/2019","7/12/2019",
             "7/13/2019","7/14/2019","7/15/2019","7/16/2019","7/18/2019","7/19/2019","7/20/2019","7/21/2019","7/30/2019",
             "7/31/2019","8/2/2019","8/3/2019","8/4/2019","8/12/2019","8/13/2019","8/14/2019","8/15/2019","8/16/2019",
             "8/17/2019","8/18/2019","8/30/2019","8/31/2019","9/1/2019","9/2/2019","9/3/2019","9/4/2019",
             "9/17/2019","9/18/2019","9/19/2019","9/20/2019","9/21/2019","9/22/2019","10/4/2019","10/5/2019","10/15/2019",
             "10/16/2019","10/17/2019","10/18/2019"]

In [22]:
# Convert data into datetime and add a boolean column
home_game_df = pd.DataFrame(home_game, columns=["HOME_GAME_DATE"])
home_game_df['DAY_OF_YEAR'] = pd.to_datetime(home_game_df["HOME_GAME_DATE"])
home_game_df['HOME_GAME'] = True
home_game_df.drop(["HOME_GAME_DATE"], axis=1, inplace=True)
home_game_df.sample(5)

,DAY_OF_YEAR,HOME_GAME
31,2019-06-02,True
7,2019-04-13,True
70,2019-09-17,True
74,2019-09-21,True
11,2019-04-18,True


In [23]:
daily_df = pd.merge(daily_df, home_game_df, how='left', on='DAY_OF_YEAR').reset_index()
daily_df.sample(5)

,level_0,index,C/A,UNIT,SCP,STATION,DATE,FIRST_ENTRIES,FIRST_EXITS,NEXT_DAY_ENTRIES,NEXT_DAY_EXITS,DAILY_ENTRIES,DAILY_EXITS,DAY_OF_YEAR,HOME_GAME_x,HAS_KIOSK,HOME_GAME_y
967349,967349,963547,R532H,R328,02-06-00,METS-WILLETS PT,05/04/2019,37867,307228,37867.0,307228.0,0.0,0.0,2019-05-04,True,True,True
245187,245187,243977,N044,R187,00-03-00,81 ST-MUSEUM,04/24/2019,8735614,13885317,8736964.0,13887744.0,1350.0,2427.0,2019-04-24,NaN,True,NaN
751783,751783,747981,R192,R039,00-00-02,MARBLE HILL-225,07/24/2019,1829252,77144,1829930.0,77165.0,678.0,21.0,2019-07-24,NaN,NaN,NaN
388059,388059,386849,N303,R015,00-00-05,5 AV/53 ST,05/29/2019,6021673,1512941,6023327.0,1513293.0,1654.0,352.0,2019-05-29,True,NaN,True
417412,417412,416202,N324,R018,00-03-02,JKSN HT-ROOSVLT,07/02/2019,6343469,2528418,6346938.0,2529807.0,3469.0,1389.0,2019-07-02,NaN,NaN,NaN


Second, we have external data for MTA digital kiosk locations. Kiosk data was obtained from the [MTA website](http://web.mta.info/nyct/OntheGoAds/MTA_Kiosk_Ridership_OTG.pdf) and manually transcribed into the following dictionary. Stations names were cross referenced with all unique station names from the original dataset to ensure our DataFrames could be merged on the "STATION" column.

In [24]:
kiosk_data = {'STATION':  ['161/YANKEE STAD', '145 ST','149/GRAND CONC','3 AV-149 ST','125 ST','96 ST','96 ST-2 AVE','METS-WILLETS PT'
                                 ,'86 ST','81 ST-MUSEUM','72 ST','72 ST-2 AVE','66 ST-LINCOLN','59 ST COLUMBUS','57 ST-7 AV','68ST-HUNTER CO'
                                 ,'59 ST','COURT SQ-23 ST','QUEENS PLAZA','61 ST WOODSIDE','74 ST-BROADWAY','FOREST HILLS 71','42 ST-PORT AUTH'
                                 ,'34 ST-HUDSON YD','34 ST-PENN STA','TIMES SQ-42 ST','34 ST-HERALD SQ','33 ST','42 ST-BRYANT PK','47-50 STS ROCK'
                                 ,'5 AVE','GRD CNTRL-42 ST','HUNTERS PT AV','23 ST','14 ST','14 ST-UNION SQ','BEDFORD AV','NASSAU AV','METROPOLITAN AV'
                                 ,'MYRTLE-WYCKOFF','ASTOR PL','METROPOLITAN AV','CHRISTOPHER ST','W 4 ST-WASH SQ','BROADWAY','BROADWAY JCT',"B'WAY-LAFAYETTE"
                                 ,'SPRING ST','PRINCE ST','CANAL ST','DELANCEY/ESSEX','RALPH AV','CHAMBERS ST','PARK PLACE','BROOKLYN BRIDGE','FULTON ST'
                                 ,'WALL ST','BOWLING GREEN','JAY ST-METROTEC','BOROUGH HALL','LAFAYETTE AV','CLINTON-WASH AV','FRANKLIN AV','ATLANTIC AV'
                                 ,'36 ST','5 AV/59 ST']
        }
kiosk_df = pd.DataFrame(kiosk_data, columns=["STATION"])
kiosk_df["HAS_KIOSK"] = True
kiosk_df.head()

,STATION,HAS_KIOSK
0,161/YANKEE STAD,True
1,145 ST,True
2,149/GRAND CONC,True
3,3 AV-149 ST,True
4,125 ST,True


In [25]:
daily_df = pd.merge(daily_df, kiosk_df, on="STATION", how="left")
daily_df.sample(5)

,level_0,index,C/A,UNIT,SCP,STATION,DATE,FIRST_ENTRIES,FIRST_EXITS,NEXT_DAY_ENTRIES,NEXT_DAY_EXITS,DAILY_ENTRIES,DAILY_EXITS,DAY_OF_YEAR,HOME_GAME_x,HAS_KIOSK_x,HOME_GAME_y,HAS_KIOSK_y
511135,503531,499729,N510,R163,02-00-01,14 ST,07/25/2019,37336,21180,3.856300e+04,2.193900e+04,1227.0,759.0,2019-07-25,NaN,True,NaN,True
951650,944046,940244,R521,R327,00-00-01,52 ST,04/25/2019,8745197,4623004,8.746301e+06,4.623833e+06,1104.0,829.0,2019-04-25,NaN,NaN,NaN,NaN
355326,352906,351696,N183,R415,00-00-00,BROAD CHANNEL,08/09/2019,76590,50328,7.670200e+04,5.040500e+04,112.0,77.0,2019-08-09,NaN,NaN,NaN,NaN
330172,327752,326542,N117,R198,01-00-02,NOSTRAND AV,09/17/2019,1024608,1463003,1.026138e+06,1.465183e+06,1530.0,2180.0,2019-09-17,True,NaN,True,NaN
722803,715199,711397,R158,R084,00-06-01,59 ST COLUMBUS,09/05/2019,1037755759,1222686022,1.037757e+09,1.222687e+09,1363.0,657.0,2019-09-05,NaN,True,NaN,True


In [26]:
# save the cleaned-up file for analysis
with open('pickled_data/yankee_game_kiosk_df.pickle', 'wb') as to_write:
    pickle.dump(daily_df, to_write)